In [6]:
import os
import cv2
import numpy as np
from tqdcm import tqdm

In [8]:
Rebuild_data=True

In [9]:
class DogsVsCats:
    img_size=50
    cats='/kaggle/input/dogs-cats-images/dog vs cat/dataset/training_set/cats/'
    dogs='/kaggle/input/dogs-cats-images/dog vs cat/dataset/training_set/dogs/'
    labels={cats:0,dogs:1}
    training_data=[]
    catcount=0
    dogcount=0
                
    def make_training_data(self):
        for label in self.labels:
            try:
                for f in tqdm(os.listdir(label)):
                    path=os.path.join(label,f)
                    img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
                    img=cv2.resize(img,(self.img_size,self.img_size))
                    self.training_data.append([np.array(img),np.eye(2)[self.labels[label]]])
                    
                    if label == self.cats:
                        self.catcount+=1
                    elif label == self.dogs:
                        self.dogcount+=1
            except Exception as e:
                pass  
            
        np.random.shuffle(self.training_data)
        np.save('training_data.npy',self.training_data)
        print('Cats:',self.catcount)
        print('Dogs:',self.dogcount)

In [10]:
if Rebuild_data:
    dogsvcats = DogsVsCats()
    dogsvcats.make_training_data()

In [11]:
training_data=np.load('training_data.npy',allow_pickle=True)

In [12]:
training_data

In [13]:
import matplotlib.pyplot as plt

plt.imshow(training_data[0][0])
plt.show()

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [31]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(1,32,5)     #f=5 - that is the frame window will be of size 5
        self.conv2=nn.Conv2d(32,64,5)
        self.conv3=nn.Conv2d(64,128,5)
        
        x=torch.randn(50,50).view(-1,1,50,50)
        self._to_linear=None
        self.convs(x)
    
    def convs(self,x):
        x=F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x=F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        x=F.max_pool2d(F.relu(self.conv3(x)),(2,2))
        
        if self._to_linear is None:
            self._to_linear=x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
    
    def forward(self,x):
        x=self.convs(x)
        x=x.view(-1,self._to_linear)
        x=F.relu(self.fc1(x))
        x=self.fc2(x)
        return F.softmax(x,dim=1)

In [32]:
net=Net()

In [33]:
net